In [5]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install scikit-learn
!pip install xgboost


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
  Using cached scikit_learn-1.6.1-cp312-cp312-macosx_12_0_arm64.whl.metadata (31 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.1-cp312-cp312-macosx_12_0_arm64.whl (11.2 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


Load data and split into training+testing data randomly (80/20 split)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Load the CSV again
df = pd.read_csv('data/brawl_scrim_040325.csv').iloc[:, 2:]

# 2. Drop the specified columns
drop_cols = [
    'Day', 'Hour',
    'P1', 'P2', 'P3', 'P4', 'P5', 'P6',
    'Tag 1', 'Tag 2', 'Tag 3', 'Tag 4', 'Tag 5', 'Tag 6',
    'Type', 'ISO', 'team 1', 'team 2'
]
df = df.drop(drop_cols, axis=1, errors='ignore')

# 3. Extract 'ID' so it's not used as a feature or target
ids = df.pop('ID')

# 4. Inspect 'Team1 Result' distribution
print("Before mapping:", df['Team1 Result'].value_counts(dropna=False))

# 5. Drop the rare 'draw' rows
df = df[df['Team1 Result'] != 'draw']

# 6. Map 'victory'->1, 'defeat'->0
df['Team1 Result'] = df['Team1 Result'].map({'victory': 1, 'defeat': 0})

# 7. Confirm everything
print("After mapping:", df['Team1 Result'].value_counts())
print("Remaining rows:", len(df))

# Split train/test data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Check remaining columns
print(df.columns.tolist())

Before mapping: Team1 Result
victory    1652
defeat     1437
draw          3
Name: count, dtype: int64
After mapping: Team1 Result
1    1652
0    1437
Name: count, dtype: int64
Remaining rows: 3089
['Mode', 'Map', 'Team1 Result', 'Brawler 1', 'Brawler 2', 'Brawler 3', 'Brawler 4', 'Brawler 5', 'Brawler 6']
        Mode             Map  Team1 Result Brawler 1 Brawler 2 Brawler 3  \
0      heist  Bridge Too Far             1     Piper     Amber     Brock   
1      heist  Bridge Too Far             0     Piper     Amber     Brock   
2  brawlBall    Pinhole Punt             1      Carl       Lou       Stu   
3  brawlBall    Pinhole Punt             1      Carl       Lou       Stu   
4      heist        Pit Stop             0     Griff  El Primo     Mr. P   

  Brawler 4 Brawler 5 Brawler 6  
0      Carl       Eve      Colt  
1      Carl       Eve      Colt  
2      Hank      Crow    Buster  
3      Hank      Crow    Buster  
4      Gray      Nita      Buzz  


Now trying xgboost

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import LabelEncoder

# Encoding text fields
print("Before encoding")
print(train_df.head())

# # list out the text columns
cat_cols = ['Mode','Map','Brawler 1','Brawler 2','Brawler 3','Brawler 4','Brawler 5','Brawler 6']
for c in cat_cols:
    train_df[c] = train_df[c].astype('category')
    test_df[c]  = test_df[c].astype('category')

# # fit a LabelEncoder for each column on training data
# encoders = {}
# for c in cat_cols:
#     le = LabelEncoder()
#     # .astype(str) in case there are any NaNs
#     train_df[c] = le.fit_transform(train_df[c].astype(str))
#     test_df[c]  = le.transform(test_df[c].astype(str))
#     encoders[c] = le
print("After encoding")
print(train_df.head())

# 1. Separate features/target
X_train = train_df.drop('Team1 Result', axis=1)
y_train = train_df['Team1 Result']
X_test  = test_df.drop('Team1 Result', axis=1)
y_test  = test_df['Team1 Result']

# 2. Define a hyperparameter grid
param_grid = {
    'max_depth':        [3, 5, 7],
    'learning_rate':    [0.01, 0.1, 0.2],
    'n_estimators':     [100, 200],
    'subsample':        [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

# 3. Loop over all combinations, train & evaluate
results = []
for params in ParameterGrid(param_grid):
    model = XGBClassifier(
        tree_method='hist',
        enable_categorical=True,
        eval_metric='logloss',
        n_jobs=-1,
        **params
    )
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    results.append({**params, 'accuracy': acc})

# 4. Collect and sort results
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('accuracy', ascending=False).reset_index(drop=True)

# 5. Inspect top performers
print(results_df.head(10))

Before encoding
         Mode             Map  Team1 Result Brawler 1 Brawler 2 Brawler 3  \
2582   bounty         Hideout             1       Gus      Carl   Melodie   
168   gemGrab       Last Stop             0       Ash        Bo     Amber   
2584   bounty         Hideout             1     Piper     Byron      Hank   
2135  gemGrab  Hard Rock Mine             1      Carl      Hank       Max   
1956  gemGrab       Last Stop             0     8-Bit     Mr. P      Carl   

     Brawler 4 Brawler 5 Brawler 6  
2582       Kit      Lola     Ollie  
168        Bea    Meeple     Ollie  
2584     Mandy       Lou       Max  
2135     Frank       Bea    Meeple  
1956   Melodie       Stu     Amber  
After encoding
         Mode             Map  Team1 Result Brawler 1 Brawler 2 Brawler 3  \
2582   bounty         Hideout             1       Gus      Carl   Melodie   
168   gemGrab       Last Stop             0       Ash        Bo     Amber   
2584   bounty         Hideout             1     Piper

/Users/adminericcheng/.pyenv/versions/3.12.4/envs/hw5/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [00:48:18] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/adminericcheng/.pyenv/versions/3.12.4/envs/hw5/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [00:48:18] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/adminericcheng/.pyenv/versions/3.12.4/envs/hw5/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [00:48:18] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/adminericcheng/.pyenv/versions/3.12.4/envs/hw5/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [00

   colsample_bytree  learning_rate  max_depth  n_estimators  subsample  \
0               0.8            0.1          7           200        0.8   
1               0.8            0.1          5           100        0.8   
2               0.8            0.1          7           100        0.8   
3               1.0            0.2          3           200        0.8   
4               1.0            0.1          3           100        0.8   
5               0.8            0.1          5           200        0.8   
6               1.0            0.2          5           100        0.8   
7               1.0            0.2          3           100        1.0   
8               1.0            0.1          5           100        0.8   
9               1.0            0.1          3           200        0.8   

   accuracy  
0  0.580906  
1  0.579288  
2  0.577670  
3  0.574434  
4  0.572816  
5  0.571197  
6  0.569579  
7  0.566343  
8  0.566343  
9  0.566343  


Increasing data set size by permuting the order of brawlers on each team! 36x increase
We do this to training/test data separately to avoid data leakage

In [ ]:
import pandas as pd
import itertools

def augment_by_swapping(df):
    """
    For each row in df, generate all permutations of Brawler 1–3 and
    Brawler 4–6, and return a new DataFrame with every permuted copy.
    """
    team1_cols = ['Brawler 1', 'Brawler 2', 'Brawler 3']
    team2_cols = ['Brawler 4', 'Brawler 5', 'Brawler 6']
    
    rows = []
    for _, row in df.iterrows():
        t1 = tuple(row[c] for c in team1_cols)
        t2 = tuple(row[c] for c in team2_cols)
        
        for perm1 in itertools.permutations(t1):
            for perm2 in itertools.permutations(t2):
                new_row = row.copy()
                # reassign permuted values
                for i, c in enumerate(team1_cols):
                    new_row[c] = perm1[i]
                for i, c in enumerate(team2_cols):
                    new_row[c] = perm2[i]
                rows.append(new_row)
    
    aug_df = pd.DataFrame(rows)
    # if any duplicate rows (e.g. brawlers repeated), drop exact duplicates:
    # aug_df = aug_df.drop_duplicates().reset_index(drop=True)
    return aug_df

# ▶ Apply separately to train and test
train_aug = augment_by_swapping(train_df)
test_aug  = augment_by_swapping(test_df)

print(f"Original train rows: {len(train_df)}, augmented: {len(train_aug)}")
print(f"Original test  rows: {len(test_df)},  augmented: {len(test_aug)}")
print(train_aug.head(5))

Original train rows: 2471, augmented: 88956
Original test  rows: 618,  augmented: 22248
        Mode      Map  Team1 Result Brawler 1 Brawler 2 Brawler 3 Brawler 4  \
2582  bounty  Hideout             1       Gus      Carl   Melodie       Kit   
2582  bounty  Hideout             1       Gus      Carl   Melodie       Kit   
2582  bounty  Hideout             1       Gus      Carl   Melodie      Lola   
2582  bounty  Hideout             1       Gus      Carl   Melodie      Lola   
2582  bounty  Hideout             1       Gus      Carl   Melodie     Ollie   
2582  bounty  Hideout             1       Gus      Carl   Melodie     Ollie   
2582  bounty  Hideout             1       Gus   Melodie      Carl       Kit   
2582  bounty  Hideout             1       Gus   Melodie      Carl       Kit   
2582  bounty  Hideout             1       Gus   Melodie      Carl      Lola   
2582  bounty  Hideout             1       Gus   Melodie      Carl      Lola   

     Brawler 5 Brawler 6  
2582      Lola 

Augmenting the data again by inverting the result and putting team2's brawler picks in team1's position. 2x the data

In [24]:
import pandas as pd

def invert_and_swap(df):
    """
    For each row in df, create a copy where:
    - Team1 and Team2 brawlers are swapped
    - Team1 Result is inverted (1->0, 0->1)
    """
    team1 = ['Brawler 1','Brawler 2','Brawler 3']
    team2 = ['Brawler 4','Brawler 5','Brawler 6']
    
    swapped = df.copy()
    swapped[team1] = df[team2].values
    swapped[team2] = df[team1].values
    swapped['Team1 Result'] = 1 - df['Team1 Result']
    return swapped

# Generate the swapped/inverted copies
train_swapped = invert_and_swap(train_aug)
test_swapped  = invert_and_swap(test_aug)

# Concatenate to double the size
train_final = pd.concat([train_aug,    train_swapped], ignore_index=True)
test_final  = pd.concat([test_aug,     test_swapped],  ignore_index=True)

print(f"Train: {len(train_aug)} → {len(train_final)} rows")
print(f"Test:  {len(test_aug)}  → {len(test_final)} rows")

print(train_final.head())

Train: 88956 → 177912 rows
Test:  22248  → 44496 rows
     Mode      Map  Team1 Result Brawler 1 Brawler 2 Brawler 3 Brawler 4  \
0  bounty  Hideout             1       Gus      Carl   Melodie       Kit   
1  bounty  Hideout             1       Gus      Carl   Melodie       Kit   
2  bounty  Hideout             1       Gus      Carl   Melodie      Lola   
3  bounty  Hideout             1       Gus      Carl   Melodie      Lola   
4  bounty  Hideout             1       Gus      Carl   Melodie     Ollie   

  Brawler 5 Brawler 6  
0      Lola     Ollie  
1     Ollie      Lola  
2       Kit     Ollie  
3     Ollie       Kit  
4       Kit      Lola  


Rerunning on augmented data set

In [29]:
cat_cols = ['Mode','Map','Brawler 1','Brawler 2','Brawler 3','Brawler 4','Brawler 5','Brawler 6']
for c in cat_cols:
    train_final[c] = train_df[c].astype('category')
    test_df[c]  = test_df[c].astype('category')

# 1. Separate features/target
X_train = train_final.drop('Team1 Result', axis=1)
y_train = train_final['Team1 Result']
X_test  = test_df.drop('Team1 Result', axis=1)
y_test  = test_df['Team1 Result']

# 2. Define a hyperparameter grid
param_grid = {
    'max_depth':        [3, 5, 7],
    'learning_rate':    [0.01, 0.1, 0.2],
    'n_estimators':     [100, 200],
    'subsample':        [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

# 3. Loop over all combinations, train & evaluate
results = []
for params in ParameterGrid(param_grid):
    model = XGBClassifier(
        tree_method='hist',
        enable_categorical=True,
        eval_metric='logloss',
        n_jobs=-1,
        **params
    )
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    results.append({**params, 'accuracy': acc})

# 4. Collect and sort results
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('accuracy', ascending=False).reset_index(drop=True)

# 5. Inspect top performers
print(results_df.head(10))

   colsample_bytree  learning_rate  max_depth  n_estimators  subsample  \
0               0.8           0.10          3           100        0.8   
1               0.8           0.20          7           100        1.0   
2               0.8           0.01          3           100        0.8   
3               1.0           0.10          3           200        0.8   
4               1.0           0.01          7           200        0.8   
5               0.8           0.10          5           100        1.0   
6               1.0           0.01          5           100        0.8   
7               1.0           0.01          3           100        0.8   
8               0.8           0.01          5           200        0.8   
9               1.0           0.20          5           200        0.8   

   accuracy  
0  0.553398  
1  0.548544  
2  0.546926  
3  0.543689  
4  0.543689  
5  0.543689  
6  0.542071  
7  0.542071  
8  0.540453  
9  0.540453  
